# Changelog

## Version 0.1 (2023-10-28)

* Initial creation of the notebook.

## Version 1.0 (2023-11-05)
- Working version of notebook to create 'clean' output file

In [ ]:
# Notebook Purpose: 
# This Jupyter notebook aims to create a clean and comprehensive dataset containing flights from Iberia and its partners. 
# It addresses several issues found in the initial dataset obtained from IB Systems, such as missing IDs, 
# inconsistent ID matching, and incomplete flying data for partner flights. 
# Additionally, the IAGL data lacks elite points information and fare product data. 
# To overcome these shortcomings, we will reverse-engineer elite points based on published Iberia tier point awarding and zones. 
# Furthermore, we will enhance partner flying data, such as British Airways (BA) and Qatar Airways, by providing more detailed information. 
# Ultimately, the goal of this worksheet is to consolidate and refine the data to ensure accuracy and reliability in all aspects.

In [86]:
_ = !pip install awswrangler -q -q
_ = !pip install dask[complete]


# Import redshift and s3 objects from awswrangler

from awswrangler import redshift, s3
import pandas as pd
from time import process_time
import numpy as np
import datetime
import math
import random
import time
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


pd.set_option('display.float_format', lambda x: '%.3f' % x)

euro = 	1.166525 #hmrc average for 2022 tax year
miles = 0.621371

iag_codes = ['BA', 'CJ', 'EI', 'EZ', 'I0', 'I2', 'IB', 'VY', 'YW']
IB_codes = ['I0', 'I2', 'IB', 'VY', 'YW']
not_EI = ['BA', 'CJ', 'EZ', 'I0', 'I2', 'IB', 'VY', 'YW', 'AA', 'AY'] ##AA included now they are in AJB
#not_EI = ['BA', 'CJ', 'EZ', 'I0', 'I2', 'IB', 'VY', 'YW', 'AY'] ##2019 version where werent in AJB

islands_airports = {
    "Palma de Mallorca": "PMI",
    "Ibiza": "IBZ",
    "Menorca": "MAH",
    "Tenerife": "TFN",
    "Tenerife":"TFE",
    "Gran Canaria": "LPA",
    "Lanzarote": "ACE",
    "Fuerteventura": "FUE",
    "La Palma": "SPC",
    "La Gomera": "GMZ",
    "El Hierro": "VDE",
    "Meliilla":"MLN",
    "Cueta":"JCU"
    
}

spanish_islands = ['Spain - Islands']

spain = ['Spain']

iberia_n_africa = [
     'Morocco', 'Algeria', 'Tunisia', 'Libya', 'Western Sahara'
]

europe_1 = ['Portugal', 'Andorra']

europe_2 = [
    'France', 'UK', 'Ireland', 'Monaco', 'Belgium', 'Luxembourg', 'Italy', 'Malta', 
    'San Marino', 'Vatican City'
]

europe_3 = [
    'Germany', 'Switzerland', 'Austria', 'Liechtenstein', 'Netherlands', 'Denmark', 
    'Czech Republic', 'Poland', 'Slovakia', 'Hungary', 'Slovenia', 'Croatia', 
    'Bosnia and Herzegovina', 'Serbia', 'Montenegro', 'North Macedonia', 'Kosovo', 
    'Albania', 'Greece', 'Romania', 'Bulgaria', 'Moldova', 'Belarus', 'Ukraine', 
    'Lithuania', 'Latvia', 'Estonia', 'Russia', 'Cyprus', 'Georgia', 'Armenia', 'Azerbaijan'
]

africa_mena_1 = [
    'Mauritania', 'Mali', 'Niger', 'Chad', 'Senegal', 'The Gambia', 'Guinea-Bissau', 
    'Guinea', 'Sierra Leone', 'Liberia', 'Côte d\'Ivoire', 'Ghana', 'Burkina Faso', 
    'Togo', 'Benin', 'Nigeria', 'Cameroon', 'Egypt', 'Israel', 'Lebanon', 'Palestine', 
    'Jordan', 'Syria', 'Iraq', 'Saudi Arabia', 'Kuwait', 'Bahrain', 'Qatar', 'United Arab Emirates', 
    'Oman', 'Yemen', 'Equatorial Guinea'
]

north_america = [
    'Greenland', 'Canada', 'USA', 'Mexico'
]

central_america_caribbean = [
    'Belize', 'Guatemala', 'El Salvador', 'Honduras', 'Nicaragua', 
    'Costa Rica', 'Panama', 'Cuba', 'Jamaica', 'Haiti', 
    'Dominican Republic', 'Puerto Rico', 'Bahamas', 'Bermuda', 
    'Barbados', 'Saint Lucia', 'Grenada', 'Saint Vincent and the Grenadines', 'Antigua and Barbuda', 
    'Saint Kitts and Nevis', 'Dominica', 'Trinidad and Tobago', 'Aruba', 'Curacao', 
    'Saint Barthelemy', 'Guadeloupe', 'Martinique', 'British Virgin Islands', 'US Virgin Islands', 
    'Anguilla', 'Saint Martin', 'Sint Maarten', 'Falkland Islands', 'South Georgia and the South Sandwich Islands'
]

africa = [
    'Central African Republic', 'South Sudan', 'Sudan', 'Eritrea', 'Djibouti', 'Somalia', 
    'Ethiopia', 'Uganda', 'Rwanda', 'Burundi', 'Kenya', 'Seychelles', 'Comoros', 'Tanzania', 
    'Zambia', 'Angola', 'Namibia', 'Botswana', 'Zimbabwe', 'South Africa', 'Lesotho', 
    'Swaziland', 'Madagascar', 'Mozambique', 'Malawi', 'Gabon', 'Republic of the Congo', 
    'Democratic Republic of the Congo', 'São Tomé and Príncipe'
]

e_asia = [
    'Turkey', 'Iran', 'Afghanistan', 'Pakistan', 'India', 'Nepal', 'Bhutan', 'Bangladesh', 
    'Sri Lanka', 'Maldives', 'Myanmar', 'Thailand', 'Laos', 'Cambodia', 'Vietnam', 'Malaysia', 
    'Singapore', 'Indonesia', 'Brunei', 'Philippines', 'Timor-Leste', 'China', 'Mongolia', 
    'North Korea', 'South Korea', 'Japan', 'Taiwan', 'Australia', 'New Zealand', 'Papua New Guinea', 
    'Fiji', 'Solomon Islands', 'Vanuatu', 'New Caledonia', 'French Polynesia', 'Wallis and Futuna', 
    'Samoa', 'American Samoa', 'Tonga', 'Tuvalu', 'Nauru', 'Kiribati', 'Marshall Islands', 'Palau', 
    'Micronesia', 'Northern Mariana Islands', 'Guam', 'Japan', 'China'
]

s_america_1 = [
    'Venezuela', 'Colombia', 'Ecuador', 'Peru','Bolivia','Guyana','Suriname'
]

s_america_2 = [
    'Chile', 'Paraguay', 'Uruguay', 'Brazil','Argentina'
]



country_to_region = {}
for country in spanish_islands:
    country_to_region[country] = 'Domestic (Inter-Island and Mainland - Melilla)'
    
for country in spain:
    country_to_region[country] = 'Spain'   
    
for country in spain:
    country_to_region[country] = 'Europe I'
    
for country in europe_2:
    country_to_region[country] = 'Europe II'

for country in europe_3:
    country_to_region[country] = 'Europe III'

for country in north_america:
    country_to_region[country] = 'North America'
    
for country in central_america_caribbean:
    country_to_region[country] = 'Central America-Caribbean'

for country in s_america_1:
    country_to_region[country] = 'South America I'

for country in s_america_2:
    country_to_region[country] = 'South America II'
    
for country in iberia_n_africa:
    country_to_region[country] = 'North Africa'

for country in africa_mena_1:
    country_to_region[country] = 'Africa I/Near East'

for country in africa:
    country_to_region[country] = 'Africa II'

for country in e_asia:
    country_to_region[country] = 'Far East'
    
cabin_mapping = {
    'J': 'Full Business (J/C/D)',
    'C': 'Full Business (J/C/D)',
    'D': 'Full Business (J/C/D)',
    
    'R': 'Reduced Business (R/I)',
    'I': 'Reduced Business (R/I)',
    'Z': 'Reduced Business (R/I)',
    'P': 'Reduced Business (R/I)',
    
    'W': 'Premium Economy (W/E/T)',
    'E': 'Premium Economy (W/E/T)',
    'T': 'Premium Economy (W/E/T)',
    
    'Y': 'Full Economy (Y/B/H)',
    'B': 'Full Economy (Y/B/H)',
    'H': 'Full Economy (Y/B/H)',
    
    'K': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'M': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'L': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'F': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'V': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'S': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'N': 'Discount Economy (K/M/L/F/V/S/N/G)',
    'G': 'Discount Economy (K/M/L/F/V/S/N/G)',
    
    'Q': 'Discount Economy (Q/O/A)',
    'O': 'Discount Economy (Q/O/A)',
    'A': 'Discount Economy (Q/O/A)',
    'X': 'Discount Economy (Q/O/A)',
}

fare_product_mapping = {
    'B': 'Basic',
    'M': 'Optima',
    'U': 'Comfort', 
    'S': 'Flexible'
}

cabin_codes = {
    "O": "Economy",
    "E": "Economy",
    "Y": "Economy",
    "M": "Economy",
    "N": "Prem Economy",
    "W": "Prem Economy",
    "T": "Prem Economy",
    "B": "Business",
    "C": "Business",
    "J": "Business",
    "D": "Business",
    "R": "Business",
    "I": "Business",
    "F": "First",
    "H": "First"
}

spanish_to_english_mapping = {
    "INTER-PENINS-MELILLA": 'Domestic (Inter-Island and Mainland - Melilla)',
    "PENINS Y BALEARES":'Spain',
    "PENINS Y CANARIAS": 'Spain',
    "ESPAÑA- EUROPA I": "Europe I",
    "ESPAÑA- EUROPA II": "Europe II",
    "ESPAÑA- EUROPA III": "Europe III",
    "ESPAÑA- NORTEAMERICA": "North America",
     "ESPAÑA- CENTROAM-CAR": "Central America-Caribbean",
    "ESPAÑA- SUDAMERICA I": "South America I",
    "ESPAÑA-SUDAMERICA II": "South America II",
    "ESPAÑA- NORTE AFRICA": "North Africa",
    "AFRICA I/PROX.ORIENT": "Africa I/Near East",
    "ESPAÑA-√ÅFRICA II": "Africa II",
    "ESP - EXTREMO ORIENT": "Far East",
    "OTROS VUELOS": "Other Flights",
}

# Define the tier points based on the IB table
points = {
    'Full Business (J/C/D)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 50,
        'Spain': 100,
        'Europe I': 100,
        'Europe II': 150,
        'Europe III': 275,
        'North Africa': 100,
        'Africa I/Near East': 350,
        'Africa II': 575,
        'North America': 575,
        'Central America-Caribbean': 575,
        'South America I': 575,
        'South America II': 700,
        'Far East': 700,
        'Other Flights':100,
    },
    'Reduced Business (R/I)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 35,
        'Spain': 75,
        'Europe I':75,
        'Europe II': 115,
        'Europe III': 225,
        'North Africa': 75,
        'Africa I/Near East': 300,
        'Africa II': 500,
        'North America': 500,
        'Central America-Caribbean': 500,
        'South America I': 500,
        'South America II':600,
        'Far East': 600,
        'Other Flights':75,
    },
    'Premium Economy (W/E/T)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 30,
        'Spain': 60,
        'Europe I': 60,
        'Europe II': 90,
        'Europe III': 175,
        'North Africa': 60,
        'Africa I/Near East': 250,
        'Africa II': 425,
        'North America': 425,
        'Central America-Caribbean': 425,
        'South America I': 425,
        'South America II': 500,
        'Far East': 500,
        'Other Flights':60,
    },
    'Full Economy (Y/B/H)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 25,
        'Spain': 50,
        'Europe I': 50,
        'Europe II': 75,
        'Europe III': 150,
        'North Africa': 50,
        'Africa I/Near East': 200,
        'Africa II': 350,
        'North America': 350,
        'Central America-Caribbean': 350,
        'South America I': 350,
        'South America II': 400,
        'Far East': 400,
        'Other Flights':50,
    },
    'Discount Economy (K/M/L/F/V/S/N/G)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 15,
        'Spain': 25,
        'Europe I': 25,
        'Europe II': 35,
        'Europe III': 75,
        'North Africa': 25,
        'Africa I/Near East': 100,
        'Africa II': 175,
        'North America': 175,
        'Central America-Caribbean': 175,
        'South America I': 175,
        'South America II':200,
        'Far East': 200,
        'Other Flights':25,
    },

    'Discount Economy (Q/O/A)': {
        'Domestic (Inter-Island and Mainland - Melilla)': 5,
        'Spain': 15,
        'Europe I': 15,
        'Europe II': 20,
        'Europe III': 35,
        'North Africa': 15,
        'Africa I/Near East': 50,
        'Africa II': 90,
        'North America': 90,
        'Central America-Caribbean': 90,
        'South America I': 90,
        'South America II':100, 
        'Far East': 100,
        'Other Flights':15,
    }
}
# Defining tier point values for other carriers
tier_points_values = {'First': [100, 150, 275, 575, 700],
                      'Business Class': [75, 115, 225, 500, 600],
                      'Full Economy': [50, 75, 150, 350, 400],
                      'Discount Economy': [15, 20, 35, 90, 100]
}

# Defining bins for mile ranges for other carriers
bins = [0, 600, 1500, 3000, 6000, np.inf]


In [112]:
def read_data_from_s3(bucket, folder, filename, dtypes, encoding='utf-16le'):
    s3_location = f's3://{bucket}/{folder}/{filename}'
    return s3.read_csv(s3_location, dtype=dtypes, low_memory=False, encoding=encoding)

def prepare_IBdf(df):
    """basic cleaning of dataframe"""
    df['key_flight_date'] = pd.to_datetime(df['key_flight_date'])
    df['pnr_creation_date'] = pd.to_datetime(df['pnr_creation_date'])

    for col in df.columns:
        if df[col].dtype == 'object': 
            df[col] = df[col].str.strip()

    df.rename(columns={
        'carrier_code':'operating_carrier_code',
        'code_class_description':'booking_cabin'
    }, inplace=True)
    
    subset_columns = [col for col in df.columns if col != 'ff_identification']
    df.drop_duplicates(subset=subset_columns, inplace=True)

    df['origins'] = 'IB'
    
    return df   

def fill_missing_checkin_ids(df):
    """
    Fills missing customer_id_checkin values with non-null customer_id_checkin 
    values from the same ticket number and customer_id, preferentially selecting 
    values that are 5 or more characters long.
    """
    
    # Helper function to get the mode, preferentially from values of length 5 or more
    def get_preferred_mode(x):
        sorted_vals = x.dropna().unique()
        sorted_vals = sorted(sorted_vals, key=lambda item: (-len(item), item))  # Sort by length and then lexically
        if len(sorted_vals) == 0:
            return np.nan
        return sorted_vals[0]

    # Group by ticket_number and customer_id and get the preferred mode for each group
    modes = df.groupby(['ticket_number', 'customer_id'])['customer_id_checkin'].transform(get_preferred_mode)
    
    # Fill NaNs in customer_id_checkin using the computed modes
    df['customer_id_checkin'].fillna(modes, inplace=True)

    return df


def correct_checkin_id(df):
    # Identify the short customer_id_checkin values
    short_ids = df[df['customer_id_checkin'].str.len() == 4]
    
    # Merge and filter matching rows
    merge_result = df.merge(df, on='ticket_number', suffixes=('', '_y'))
    merge_result = merge_result[
        (merge_result['customer_id_checkin'].str.len() == 4) &
        (merge_result['customer_id_checkin_y'].str.contains(merge_result['customer_id_checkin'])) &
        (merge_result['customer_id_checkin'] != merge_result['customer_id_checkin_y'])
    ]
    
    # For rows with a matching longer checkin id, replace the short id in the original dataframe
    for idx, row in merge_result.iterrows():
        mask = (df['ticket_number'] == row['ticket_number']) & (df['customer_id_checkin'] == row['customer_id_checkin'])
        df.loc[mask, 'customer_id_checkin'] = row['customer_id_checkin_y']

    return df
                
# Function to extract cabin name from booking_class for rows where booking_cabin is 'Economy'
def extract_cabin_name(row):
    if row['booking_cabin'] == 'Economy':
        if 'Discount Economy' in row['booking_class']:
            return 'Discount Economy'
        elif 'Full Economy' in row['booking_class']:
            return 'Full Economy'
    return row['booking_class']

def get_points_from_class_zone(class_zone):
    if not isinstance(class_zone, tuple):
        return None

    booking_class, route_name = class_zone
    return points.get(booking_class, {}).get(route_name, None)

def assign_base_and_destination(IBdf, islands_airports, country_to_region):
    """
    Vectorize the assignment of base and destination for calculation of tier points
    """
    # Identify routes associated with Spanish islands and enclaves
    mask_island_1 = IBdf['STN_1'].isin(islands_airports.values())
    mask_island_2 = IBdf['STN_2'].isin(islands_airports.values())

    # Create necessary masks for np.where conditions
    mask_spain_island_1 = (IBdf['CTRY_NM_1'] == "Spain") & mask_island_1
    mask_spain_island_2 = (IBdf['CTRY_NM_2'] == "Spain") & mask_island_2
    mask_both_islands = mask_island_1 & mask_island_2
    mask_spain_1 = IBdf['CTRY_NM_1'] == "Spain"
    mask_spain_2 = IBdf['CTRY_NM_2'] == "Spain"

    # Define base_country and destination assignment logic
    IBdf['base_country'] = np.where(
        mask_spain_island_1, 'Spain - Islands',
        np.where(mask_spain_island_2, 'Spain - Islands',
                 np.where(mask_island_1, IBdf['CTRY_NM_1'],
                          np.where(mask_island_2, IBdf['CTRY_NM_2'],
                                   np.where(mask_spain_1, IBdf['CTRY_NM_1'],
                                            np.where(mask_spain_2, IBdf['CTRY_NM_2'], IBdf['CTRY_NM_1']))))))

    IBdf['destination'] = np.where(
        mask_spain_island_1, IBdf['CTRY_NM_2'],
        np.where(mask_spain_island_2, IBdf['CTRY_NM_1'],
                 np.where(mask_both_islands, 'Spain - Islands',
                          np.where(mask_island_1, IBdf['CTRY_NM_2'],
                                   np.where(mask_island_2, IBdf['CTRY_NM_1'],
                                            np.where(mask_spain_1, IBdf['CTRY_NM_2'],
                                                     np.where(mask_spain_2, IBdf['CTRY_NM_1'], IBdf['CTRY_NM_2'])))))))
    
    mask = IBdf['base_country'].isin(['Spain', 'Spain - Islands'])
    IBdf.loc[mask, 'dest_region'] = IBdf.loc[mask, 'destination'].map(country_to_region)
    IBdf['route_name'] = IBdf['route_name'].fillna(IBdf['dest_region'].fillna('Other Flights'))
    
    return IBdf

def create_matches_and_best_id_columns(IBdf):
    """ create a 'matches' to say where customer id originated and fill in customer id best with criteria specified preferring customer id checkin"""
    # Define the conditions and choices
    conditions = [
        (IBdf['customer_id'] == IBdf['customer_id_checkin']),
        (IBdf['customer_id_checkin'].str.len() < 5),
        (~IBdf['customer_id_checkin'].isnull()) & (IBdf['customer_id'].isnull()),
        (~IBdf['customer_id'].isnull()) & (IBdf['customer_id_checkin'].isnull()),
        (IBdf['customer_id'].isnull()) & (IBdf['customer_id_checkin'].isnull())
    ]

    choices_matches = [
        'Completely matching',
        'customer_id_checkin too short, filled with booking id',
        'customer_id_bookings is null filled with checkin id',
        'customer_id_checkin is null filled with booking id',
        'Both are null, flight will be deleted'
    ]

    choices_best_id = [
        IBdf['customer_id'],
        IBdf['customer_id'],
        IBdf['customer_id_checkin'],
        IBdf['customer_id'],
        np.nan
    ]

    # Assign values to columns based on the conditions and choices
    IBdf['matches'] = np.select(conditions, choices_matches, default='Unknown')
    IBdf['customer_id_best'] = np.select(conditions, choices_best_id, default=np.nan)

    return IBdf

def get_points_from_class_zone(class_zone):
    """using points dictionary and IB tier point awarding table compute tier points"""
    if not isinstance(class_zone, tuple):
        return None

    booking_class, route_name = class_zone
    return points.get(booking_class, {}).get(route_name, None)

def compute_tier_points_other_carriers(row):
    """computing tier points based on mileage for select non IB carriers"""
    miles = row['miles']
    cabin = row['cabin_awarding']
    if cabin in tier_points_values and row['tier_points'] is np.nan and row['mask_included']:
        return tier_points_values[cabin][np.digitize(miles, bins) - 1]
    return row['tier_points']

def custom_join(left_df, right_df, join_cols, indicator):
    """Perform an outer join and then fill and drop _x and _y columns."""
    
    # Outer join
    join_df = left_df.merge(right_df, on=join_cols, how='outer', indicator=indicator)
    
    # Split the joined dataframe based on the indicator column
    left_only = join_df[join_df[indicator] == 'left_only'].copy()
    right_only = join_df[join_df[indicator] == 'right_only'].copy()
    join_df = join_df[join_df[indicator] == 'both'].copy()

    # Drop the indicator column from the split dataframes
    left_only.drop(columns=[indicator], inplace=True)
    right_only.drop(columns=[indicator], inplace=True)
    join_df.drop(columns=[indicator], inplace=True)

    # Handle left_only: Keep only _x columns and rename
    left_cols_to_drop = [col for col in left_only if col.endswith('_y')]
    left_only.drop(columns=left_cols_to_drop, inplace=True)
    left_rename_map = {col: col.rstrip('_x') for col in left_only.columns if col.endswith('_x')}
    left_only.rename(columns=left_rename_map, inplace=True)

    # Handle right_only: Keep only _y columns and rename
    right_cols_to_drop = [col for col in right_only if col.endswith('_x')]
    right_only.drop(columns=right_cols_to_drop, inplace=True)
    right_rename_map = {col: col.rstrip('_y') for col in right_only.columns if col.endswith('_y')}
    right_only.rename(columns=right_rename_map, inplace=True)

    # For join_df, update _x values using _y values where necessary
    x_cols = [col for col in join_df if col.endswith('_x')]
    y_cols = [col.replace('_x', '_y') for col in x_cols if col.replace('_x', '_y') in join_df]
    
    # Create dataframes for x and y columns
    df_x = join_df[x_cols].copy()
    df_y = join_df[y_cols].copy()

    # Ensure both df_x and df_y have the same dtypes
    for x_col, y_col in zip(x_cols, y_cols):
        if df_x[x_col].dtype != df_y[y_col].dtype:
            common_type = np.find_common_type([df_x[x_col].dtype, df_y[y_col].dtype], [])
            df_x[x_col] = df_x[x_col].astype(common_type)
            df_y[y_col] = df_y[y_col].astype(common_type)

    # Update _x values if they are 'Unknown'/string nan/0 using _y values
    mask = (df_x == 'Unknown') | (df_x == 'nan') | (df_x == 0) | (df_x == np.nan)
    df_x = df_x.where(~mask, df_y)

    # Update NaN values in _x using _y values
    df_x.update(df_y)
    join_df[x_cols] = df_x  # Assign the updated df_x values to joined_df
    join_df.drop(columns=y_cols, inplace=True)
    rename_map = {x: x.rstrip('_x') for x in x_cols}
    join_df.rename(columns=rename_map, inplace=True)

    return join_df, left_only, right_only



def fill_missing_miles(df):
    """filling in missing/zero miles where exists for the route in question from routes_v7"""
    bucket = 'gdp-dev-iagl-data-sagemaker'
    folder = 'PHXTierModelling/2023_data'
    filename = 'route_2023v7.csv'
    
    s3_location = f's3://{bucket}/{folder}/{filename}'
    routes = s3.read_csv(s3_location,low_memory=False)
    routes = routes.rename(columns={"Route": "route"})
    merged_df = df.merge(routes[['route', 'distanceMiles']], on='route', how='left')
    mask = (merged_df['miles'].isna()) | (merged_df['miles'] == 0)  # identify blank or zero miles
    merged_df.loc[mask, 'miles'] = merged_df.loc[mask, 'distanceMiles']  # replace with distanceMiles
    df_updated = merged_df.drop(columns=['distanceMiles'])
    
    return df_updated

def replace_values(base_df, fill_df, merge_cols):       
    merged_df = base_df.merge(fill_df[['agl_party_id'] + merge_cols], 
                              on=merge_cols, 
                              how='left',
                              suffixes=('', '_fill'))
    
    # Where there's a non-NA value in 'agl_party_id_fill', overwrite 'agl_party_id' with that value
    mask = ~merged_df['agl_party_id_fill'].isna()
    merged_df.loc[mask, 'agl_party_id'] = merged_df.loc[mask, 'agl_party_id_fill']
    merged_df.drop('agl_party_id_fill', axis=1, inplace=True)
   
    fill_and_drop_x_y_columns(merged_df)
    
    return merged_df

def replace_values_route(base_df, fill_df, merge_cols):       
    merged_df = base_df.merge(fill_df[['route'] + merge_cols], 
                              on=merge_cols, 
                              how='left',
                              suffixes=('', '_fill'))
    
    # Where there's a non-NA value in 'agl_party_id_fill', overwrite 'agl_party_id' with that value
    mask = ~merged_df['route_fill'].isna()
    merged_df.loc[mask, 'route'] = merged_df.loc[mask, 'route']
    merged_df.drop('route_fill', axis=1, inplace=True)
   
    fill_and_drop_x_y_columns(merged_df)
    
    return merged_df

def vectorized_fare_class(df):
    # Create a mask for NaN values to handle them later
    nan_mask = df['fare_basis'].isna()
    df['fare_basis'].fillna('######', inplace=True)
    df['fare_basis'] = df['fare_basis'].str.pad(4, side='left', fillchar='#')
    
    # Extract the second last and fourth last characters
    second_last = df['fare_basis'].str[-2]
    fourth_last = df['fare_basis'].str[-4]
    
    # Conditions
    cond1 = ~second_last.isin(['B', 'M', 'U', 'S'])
    cond2 = ~fourth_last.isin(['B', 'M', 'U', 'S'])
    
    # Apply conditions using numpy's where function
    result = np.where(cond1, np.where(cond2, second_last, fourth_last), second_last)
    
    # Replace the results for originally NaN values with NaN or a default value
    result[nan_mask] = np.nan
    
    # Remove padding from the original fare_basis column
    df['fare_basis'] = df['fare_basis'].str.replace('#', '')
    
    return result


def fill_and_drop_x_y_columns(joined_df):
    """Fill _x columns with _y values and drop _y columns after joins"""

    # Identify _x and _y columns
    x_cols = [col for col in joined_df if col.endswith('_x')]
    y_cols = [col.replace('_x', '_y') for col in x_cols if col.replace('_x', '_y') in joined_df]

    # Create dataframes for x and y columns
    df_x = joined_df[x_cols].copy()  
    df_y = joined_df[y_cols].copy()

    # Ensure both df_x and df_y have the same dtypes
    for x_col, y_col in zip(x_cols, y_cols):
        if df_x[x_col].dtype != df_y[y_col].dtype:
            common_type = np.find_common_type([df_x[x_col].dtype, df_y[y_col].dtype], [])
            df_x[x_col] = df_x[x_col].astype(common_type)
            df_y[y_col] = df_y[y_col].astype(common_type)

    # Update _x values if they are 'Unknown'/string nan/0 using _y values
    mask = (df_x == 'Unknown') | (df_x == 'nan') | (df_x == 0) | (df_x == np.nan)
    df_x = df_x.where(~mask, df_y)

    # Update NaN values in _x using _y values
    df_x.update(df_y)
    joined_df[x_cols] = df_x  # Assign the updated df_x values to joined_df
    joined_df.drop(columns=y_cols, inplace=True)
    rename_map = {x: x.rstrip('_x') for x in x_cols}
    joined_df.rename(columns=rename_map, inplace=True)

In [3]:
bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data'
filename = 'IDfile.csv'

dtypes = {'agl_party_id': object, 'membership_txt': object, 'scheme': object,'country_cd':object,
          'market':object,'cust_birth_date':object,'cust_join_date':object,'gender':object}

s3_location = f's3://{bucket}/{folder}/{filename}'
IDdf = s3.read_csv(s3_location,dtype=dtypes)

IDdf['cust_birth_date'] = pd.to_datetime(IDdf['cust_birth_date'],errors='coerce')
IDdf['cust_join_date'] = pd.to_datetime(IDdf['cust_join_date'],errors='coerce')

mask = IDdf['scheme']=='IBPLU'
IBIDdf = IDdf[mask]

In [4]:
#################### this is iagl agl flow avios collection data####################

bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'


dtypes = {
    'agl_party_id': 'object',
    'SRC_PARTY_ID': 'object',
    'ticket_number': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'fare': 'float64',
    'yq': 'float64',
    'tax': 'float64',
    'total_payment': 'float64',
    'unified_rev':'float64',
    'ELIGIBLE_REV_VLU':'float64',
    'channel': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'island_fares': 'object',
    'marketing_carrier_code': 'object',
    'carrier_code': 'object',
    'seg_haul_segment': 'object',
    'tier': 'object',
    'avios_earned': 'float',
    'avios_earned_prom': 'object',
    'tier_points': 'object',
    'avios_redeemed': 'object',
    'avios_redeemed_prom': 'object',
    'ff_tier_level': 'object',
    'miles': 'float64',
    'psjs': 'object'
}


filename = 'IAGL_flights.csv'
s3_location = f's3://{bucket}/{folder}/{filename}'
df = s3.read_csv(s3_location,dtype=dtypes,low_memory=False)

# If all columns are of string type
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# If not sure about column types
for col in df.columns:
    if df[col].dtype == 'object': 
        df[col] = df[col].str.strip()

########### join id file to get other customer info such as birthdate and memership txt
df['agl_party_id']=df['agl_party_id'].astype(str)
df =pd.merge(IBIDdf, df, how='right',on='agl_party_id')

condition = df['ELIGIBLE_REV_VLU'] != df['unified_rev']
df['accurate_exchange'] = df['unified_rev']/df['ELIGIBLE_REV_VLU']
df.loc[condition, 'fare'] = (df.loc[condition, 'fare'] * df.loc[condition, 'accurate_exchange']).round(2) #extracted exch rte but it's only 5dp so doesn't work for pesos
df.loc[condition, 'tax'] = (df.loc[condition, 'tax'] * df.loc[condition, 'accurate_exchange']).round(2)
df['unified_rev'] = df['unified_rev'].fillna(df['fare'])

df['carrier_no'] = df['ticket_number'].str[:-10]
df['ticket_number'] = df['ticket_number'].str[-10:]
df['key_flight_date'] = pd.to_datetime(df['key_flight_date'])
df['pnr_creation_date'] = pd.to_datetime(df['pnr_creation_date'])
################### vueling have made up ticket numbers below for some reason
df.loc[df['ticket_number'].isin(['1111111111', '1111111112', '1111111122', '1111111222', '1111111113','0000000000']), 'ticket_number'] = np.nan
df.loc[df['ticket_number'].isin([1111111111, 1111111112, 1111111122, 1111111222, 1111111113,0000000000]), 'ticket_number'] = np.nan
df.loc[df['ticket_number'].astype(str).str.len() <= 3, 'ticket_number'] = np.nan
# Placeholder
placeholder = -99999
df['ticket_number'].fillna(placeholder, inplace=True)

####### coupon number is the ordr of flights on a ticket and tx_id also follows this so make a fake coupon number to match somewhat IB one, not sure if I will use yet
df = df.sort_values(by=['ticket_number', 'pnr', 'key_flight_date', 'TX_ID'])
mask = df['pnr'].str.len() >= 5
df['coupon_number'] = df.groupby('ticket_number').cumcount().add(1).where(mask, np.nan)


# Reverting placeholder to NaN
df['ticket_number'].replace(placeholder, np.nan, inplace=True)

df['origins'] = 'IAGL'


df['booking_class_letter'] = df['booking_class']
df['booking_cabin_letter'] = df['booking_cabin'] 
(df[['booking_cabin_letter','booking_class_letter']].value_counts()).to_csv('classes.csv')
df['booking_cabin'] = df['booking_cabin_letter'].map(cabin_codes)
(df[['booking_cabin','booking_class_letter']].value_counts()).to_csv('figureoutclasses.csv')
df['booking_class'] = df['booking_class_letter'].map(cabin_mapping)

# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename ='routemapping_IB.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
IBroutes = s3.read_csv(s3_location)

df = df.merge(IBroutes, how='left', on='route')

########### tier points reverse engineering############
# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data'
filename = 'phoenixregions.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
regions = s3.read_csv(s3_location)

# Extract STN_1 and STN_2 from the route
routes = df[['route']].drop_duplicates()
routes['STN_1'] = routes['route'].str[:3]
routes['STN_2'] = routes['route'].str[3:]
# Merge with regions for STN_1
merged_stn1 = routes.merge(regions[['STN_CD','CTRY_NM']], how='left', left_on='STN_1', right_on='STN_CD')
merged_stn1 = merged_stn1.rename(columns={'CTRY_NM': 'CTRY_NM_1'})
merged_stn1.drop(['STN_CD'], axis=1, inplace=True)
# Merge with regions for STN_2
merged_stn2 = routes.merge(regions[['STN_CD','CTRY_NM']], how='left', left_on='STN_2', right_on='STN_CD')
merged_stn2 = merged_stn2.rename(columns={'CTRY_NM': 'CTRY_NM_2'})
merged_stn2.drop(['STN_CD'], axis=1, inplace=True)
# Combine the two merged dataframes
phoenixroutes = pd.concat([merged_stn1.set_index('route'), merged_stn2[['route', 'CTRY_NM_2']].set_index('route')], axis=1, join='inner').reset_index()
# Merge the resulting phoenixroutes DataFrame with the main df DataFrame
df = df.merge(phoenixroutes[['STN_1','STN_2','CTRY_NM_1', 'CTRY_NM_2', 'route']], on='route', how='left')

df = assign_base_and_destination(df, islands_airports, country_to_region)  
    
mask = df['base_country'].isin(['Spain', 'Spain - Islands'])

df.loc[mask, 'dest_region'] = df.loc[mask, 'destination'].map(country_to_region)
df['route_name'] = df['route_name'].fillna(df['dest_region'].fillna('Other Flights'))

columns_to_drop = ['STN_1', 'STN_2', 'CTRY_NM_1', 'CTRY_NM_2', 'base_country', 'destination']
df = df.drop(columns=columns_to_drop)

########### tier points reverse engineering############

##################### still working on this, haven't got fare class classification from IB ###################
######## https://www.iberia.com/gb/iberiaplus/earn-elite-points/
cols_to_fill = ['booking_class', 'route_name', 'operating_carrier_code', 'marketing_carrier_code', 'booking_cabin']
df[cols_to_fill] = df[cols_to_fill].fillna('Unknown')
     

# Map the class_zone to points
df['class_zone'] = list(zip(df['booking_class'], df['route_name']))
df['tier_points'] = np.nan

mask_operating = (df['operating_carrier_code'].isin(['IB', 'I2', 'YW','VY']))                                             
mask = ((mask_operating) & (df['marketing_carrier_code'].isin(['IB', 'I2', 'YW', 'VY'])) & (df['booking_class'] != 'Unknown') & (df['route_name'] != 'Unknown'))
df.loc[mask, 'tier_points'] = df.loc[mask, 'class_zone'].apply(get_points_from_class_zone)


############### assign tier points for other carriers ######################### https://www.iberia.com/gb/iberiaplus/earn-elite-points/
mask_included = ((df['operating_carrier_code'].isin(['NT', 'VY', 'AA', 'BA','CX','AY','JL','MH','QF','QR','AT','I0'])) &
                 (df['marketing_carrier_code'].isin(['IB', 'I2', 'YW', 'VY','NT', 'VY', 'AA', 'BA','CX','AY','JL','MH','QF','QR','AT','I0'])))

df = fill_missing_miles(df) #fill in missing miles value if they exist for route

# Apply the function to create the cabin_awarding column
df['cabin_awarding'] = df.apply(extract_cabin_name, axis=1)
df['tier_points'] = df.apply(compute_tier_points_other_carriers, axis=1)
df['tier_points'] = df['tier_points'].fillna(0)
df['psjs'] = 1

df.rename(columns={'Tier':'ff_tier_level'}, inplace=True)


for column in df:
    if df[column].dtype == "float64":
        df[column]=pd.to_numeric(df[column], downcast="float")
    if df[column].dtype == "int64":
        df[column]=pd.to_numeric(df[column], downcast="integer")

bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IAGLprocessed.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
s3.to_csv(df, s3_location, index=False)

{'paths': ['s3://gdp-dev-iagl-data-sagemaker/PHXTierModelling/2023_data/IB_dirty/IAGLprocessed.csv'],
 'partitions_values': {}}

In [52]:
####################################################################################################
# IB DATA PROCESSING NOTES:
# - IB data contains unreliable customer IDs.
#     - booking_id represents customer_id.
#     - checkin_id represents customer_id_checkin.
#     - Preferentially, checkin_id is selected since the person flying accrues points.
# - However, checkin_id has many concatenated rows. For instance, a single ticket might have 
#   checkin_id values like 1234 and 123456789, with the longer version being accurate.
#     - Wherever shorter IDs are contained within longer IDs, choose the longer ID.
# - If checkin_id is absent or too short, replace it with booking_id.
# - The data should be joined to the ID file to retrieve the agl party id.
# - Other issues with IB data:
#     - agl party ids sometimes do not align with IAGL rows due to the aforementioned ID issues.
#     - The IB dataset contains inconsistent or missing data for many attributes like route, PNR, 
#       operating/marketing carrier, avios, tier points, etc.
# - Solution:
#     - Perform a series of joins to fill gaps, starting with the most reliable joins.
#     - As joins progress and become less certain, continue to fill any null values in the original
#       IB data with available IAGL data.
#    - joins not in this code are dubious so am still checking if they are okay/will work and regularly writing out files towards the end before final decision
####################################################################################################
######### each ticket number has a row for each flight on booking and for each customer, pnr and ticket_number and similar in that regard
######### pnr is better populated in iagl data (expecially for other carriers), ticekt_number better populated in IB data
                

bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IB_EQ_2022.csv'

dtypes = {
    'customer_id': 'object',
    'customer_id_checkin': 'object',
    'ff_identification': 'object',
    'ticket_number': 'int64',
    'coupon_number': 'object',
    'fare_family_description': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'zone_description': 'object',
    'shuttle_indicator': 'object',
    'fare': 'float64',
    'yq': 'float64',
    'total_payment': 'float64',
    'channel': 'object',
    'fare_basis': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'seg_haul_segment': 'object',
    'avios_earned': 'object',
    'avios_earned_prom': 'object',
    'ff_tier_level': 'object',
    'tier_points': 'float64',
    'avios_redeemed': 'object',
    'avios_redeemed_prom': 'object',
    'booking_cabin': 'object',
    'miles': 'float64',
    'pnr': 'object',
    'pnr_creation_date': 'object',
    'psjs': 'object',
}
print('reading original IB extract')
s3_location = f's3://{bucket}/{folder}/{filename}'
IBdf = read_data_from_s3(bucket, folder, filename, dtypes)
print('tidy data')
IBdf = prepare_IBdf(IBdf)

# If not sure about column types
for col in IBdf.columns:
    if IBdf[col].dtype == 'object': 
        IBdf[col] = IBdf[col].str.strip()

# Start the timer
start_time = time.time()
# Convert necessary columns to string for easier comparison
print(IBdf['customer_id_checkin'].isna().sum())
print('fill null checkin ids where a checkin id exists on that ticket has a and booking id matches')
IBdf = fill_missing_checkin_ids(IBdf)
print('assign customer id best and where came from in matches column')
IBdf = create_matches_and_best_id_columns(IBdf)
IBdf['customer_id_best'] = IBdf['customer_id_best'].str.zfill(14)

print(IBdf.shape)

IBdf = pd.merge(IBIDdf, IBdf, how='right', left_on=['membership_txt'], right_on=['customer_id_best'], indicator=True)

print('fill in fare class, booking cabin,booking class')
# Use the vectorized function
IBdf['fare_class'] = vectorized_fare_class(IBdf)
IBdf['fare_product_name'] = IBdf['fare_class'].map(fare_product_mapping)
IBdf['booking_class_letter'] = IBdf['fare_basis'].str[0]
IBdf['booking_class'] = IBdf['booking_class_letter'].map(cabin_mapping)
IBdf['booking_cabin_letter'] = IBdf['booking_cabin'] 
IBdf['booking_cabin'] = IBdf['booking_cabin_letter'].map(cabin_codes)
IBdf['route_name'] = IBdf['zone_description'].replace(spanish_to_english_mapping)

########### tier points reverse engineering############
##################### still working on this, haven't got fare class classification from IB ###################
# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data'
filename = 'phoenixregions.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
regions = s3.read_csv(s3_location)

# Extract STN_1 and STN_2 from the route
IBdf['STN_1'] = IBdf['route'].str[:3]
IBdf['STN_2'] = IBdf['route'].str[3:]

# Merge with regions for STN_1
IBdf = IBdf.merge(
    regions[['STN_CD', 'CTRY_NM']], 
    left_on='STN_1', 
    right_on='STN_CD', 
    how='left'
).rename(columns={'CTRY_NM': 'CTRY_NM_1'}).drop('STN_CD', axis=1)

# Merge with regions for STN_2
IBdf = IBdf.merge(
    regions[['STN_CD', 'CTRY_NM']], 
    left_on='STN_2', 
    right_on='STN_CD', 
    how='left'
).rename(columns={'CTRY_NM': 'CTRY_NM_2'}).drop('STN_CD', axis=1)


IBdf = assign_base_and_destination(IBdf, islands_airports, country_to_region)

########### tier points reverse engineering############
######## https://www.iberia.com/gb/iberiaplus/earn-elite-points/
IBdf['booking_class'].fillna('Unknown', inplace=True)
IBdf['route_name'].fillna('Unknown', inplace=True)
IBdf['operating_carrier_code'].fillna('Unknown', inplace=True)
IBdf['marketing_carrier_code'].fillna('Unknown', inplace=True)
IBdf['booking_cabin'].fillna('Unknown', inplace=True)
     
print('mapping booking class and IB region')
# Map the class_zone to points
IBdf['class_zone'] = list(zip(IBdf['booking_class'], IBdf['route_name']))
operating_codes = ['IB', 'I2', 'YW', 'VY']
marketing_codes = operating_codes
mask_operating = IBdf['operating_carrier_code'].isin(operating_codes)
mask_marketing = IBdf['marketing_carrier_code'].isin(marketing_codes)

mask_conditions = mask_operating & mask_marketing & (IBdf['booking_class'] != 'Unknown') & (IBdf['route_name'] != 'Unknown') & (pd.isnull(IBdf['tier_points']))                             
print('assigning IB tier points')
IBdf.loc[mask_conditions, 'tier_points'] = IBdf.loc[mask_conditions, 'class_zone'].map(get_points_from_class_zone)

############### assign tier points for other carriers ######################### https://www.iberia.com/gb/iberiaplus/earn-elite-points/
mask_included = ((df['operating_carrier_code'].isin(['NT', 'VY', 'AA', 'BA','CX','AY','JL','MH','QF','QR','AT','I0'])) &
                 (df['marketing_carrier_code'].isin(['IB', 'I2', 'YW', 'VY','NT', 'VY', 'AA', 'BA','CX','AY','JL','MH','QF','QR','AT','I0'])))

# Apply the function to create the cabin_awarding column
IBdf['booking_cabin'].fillna('Unknown', inplace=True)
IBdf['cabin_awarding'] = IBdf.apply(extract_cabin_name, axis=1)
print('assigning other carrier tier points')
IBdf = fill_missing_miles(IBdf) #fill in missing miles value if they exist for route
IBdf['tier_points'] = IBdf.apply(compute_tier_points_other_carriers, axis=1)

IBdf['tier_points'] = IBdf['tier_points'].fillna(0)

IBdf['unified_rev'] = IBdf['fare'] + IBdf['yq']


print('saving to s3')

for column in IBdf:
    if IBdf[column].dtype == "float64":
        IBdf[column]=pd.to_numeric(IBdf[column], downcast="float")
    if IBdf[column].dtype == "int64":
        IBdf[column]=pd.to_numeric(IBdf[column], downcast="integer")
        
bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IBprocessed.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
s3.to_csv(IBdf, s3_location, index=False)


reading original IB extract
tidy data
2053416
fill null checkin ids where a checkin id exists on that ticket has a and booking id matches
assign customer id best and where came from in matches column
(7565701, 37)
fill in fare class, booking cabin,booking class
mapping booking class and IB region
assigning IB tier points
assigning other carrier tier points
saving to s3


{'paths': ['s3://gdp-dev-iagl-data-sagemaker/PHXTierModelling/2023_data/IB_dirty/IBprocessed.csv'],
 'partitions_values': {}}

In [94]:
# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IBprocessed.csv'

dtypes = {
    'agl_party_id': 'object',
    'membership_txt': 'object',
    'scheme': 'object',
    'country_cd': 'object',
    'market': 'object',
    'cust_birth_date': 'object',
    'cust_join_date': 'object',
    'gender': 'object',
    'customer_id': 'object',
    'customer_id_checkin': 'object',
    'ff_identification': 'object',
    'ticket_number': 'object',
    'coupon_number': 'float64',
    'fare_family_description': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'zone_description': 'object',
    'shuttle_indicator': 'object',
    'fare': 'float32',
    'yq': 'float32',
    'total_payment': 'float32',
    'channel': 'object',
    'fare_basis': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'seg_haul_segment': 'object',
    'avios_earned': 'float64',
    'avios_earned_prom': 'object',
    'ff_tier_level': 'object',
    'tier_points': 'float32',
    'avios_redeemed': 'object',
    'avios_redeemed_prom': 'object',
    'booking_cabin': 'object',
    'miles': 'float32',
    'pnr': 'object',
    'pnr_creation_date': 'object',
    'psjs': 'int64',
    'origins': 'object',
    'matches': 'object',
    'customer_id_best': 'object',
    '_merge': 'category',
    'fare_class': 'object',
    'fare_product_name': 'object',
    'booking_class_letter': 'object',
    'booking_class': 'object',
    'booking_cabin_letter': 'object',
    'route_name': 'object',
    'STN_1': 'object',
    'STN_2': 'object',
    'CTRY_NM_1': 'object',
    'CTRY_NM_2': 'object',
    'base_country': 'object',
    'destination': 'object',
    'dest_region': 'object',
    'class_zone': 'object',
    'cabin_awarding': 'object',
    'unified_rev': 'float32'
}


s3_location = f's3://{bucket}/{folder}/{filename}'
IBdf = s3.read_csv(s3_location,dtype=dtypes)
# Convert key_flight_date to datetime
IBdf['key_flight_date'] = pd.to_datetime(IBdf['key_flight_date'])
IBdf['pnr_creation_date'] = pd.to_datetime(IBdf['pnr_creation_date'])
IBdf['cust_birth_date'] = pd.to_datetime(IBdf['cust_birth_date'],errors='coerce')
IBdf['cust_join_date'] = pd.to_datetime(IBdf['cust_join_date'],errors='coerce')

IBdf = IBdf[['agl_party_id',
 'membership_txt',
 'customer_id_best',
  'ff_tier_level',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',        
 'ticket_number',
'pnr',
 'pnr_creation_date',
 'coupon_number',
 'fare_family_description',
 'key_flight_date',
 'route',
 'fare',
 'yq',
 'total_payment',
 'unified_rev',
 'channel',
 'fare_basis',
 'marketing_carrier_code',
 'operating_carrier_code',
 'seg_haul_segment',
 'avios_earned',
 'tier_points',
 'avios_redeemed',
 'booking_cabin',
 'miles',
 'fare_class',
 'fare_product_name',
 'booking_class_letter',
 'booking_class',
 'booking_cabin_letter',
'zone_description',
 'shuttle_indicator',
 'route_name',
 'class_zone',
  'corp_fares',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'psjs']]

In [96]:
# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IAGLprocessed.csv'

dtypes = {
    'agl_party_id': 'object',
    'membership_txt': 'object',
    'scheme': 'object',
    'country_cd': 'object',
    'market': 'object',
    'cust_birth_date':  'object',
    'cust_join_date':  'object',
    'gender': 'object',
    'SRC_PARTY_ID': 'object',
    'TX_ID': 'object',
    'ticket_number': 'float64',
    'pnr': 'object',
    'pnr_creation_date':  'object',
    'key_flight_date':  'object',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'route': 'object',
    'booking_cabin': 'object',
    'booking_class': 'object',
    'fare': 'float32',
    'tax': 'float32',
    'ELIGIBLE_REV_VLU': 'float32',
    'unified_rev': 'float32',
    'AWARD_CCY_CD': 'object',
    'EXCH_RTE': 'float32',
    'MULTIPLIER_NO': 'float32',
    'Tier': 'object',
    'JB_IND': 'object',
    'DISCOUNT_ECONOMY_IND': 'float32',
    'miles': 'float32',
    'seg_haul_segment': 'object',
    'FARE_BASIS_CD': 'float32',
    'BILLING_METHODOLOGY_NM': 'object',
    'FARE_TYP': 'object',
    'SBE_REGION_TXT': 'object',
    'accurate_exchange': 'float32',
    'carrier_no': 'object',
    'coupon_number': 'float64',
    'origins': 'object',
    'booking_class_letter': 'object',
    'booking_cabin_letter': 'object',
    'zone_description': 'object',
    'route_name': 'object',
    'dest_region': 'object',
    'class_zone': 'object',
    'tier_points': 'float32',
    'avios_earned': 'float64',
    'cabin_awarding': 'object'
}


s3_location = f's3://{bucket}/{folder}/{filename}'
df = s3.read_csv(s3_location,dtype=dtypes,low_memory=False)
# Convert key_flight_date to datetime
df['key_flight_date'] = pd.to_datetime(df['key_flight_date'])
df['pnr_creation_date'] = pd.to_datetime(df['pnr_creation_date'])
df['cust_birth_date'] = pd.to_datetime(df['cust_birth_date'],errors='coerce')
df['cust_join_date'] = pd.to_datetime(df['cust_join_date'],errors='coerce')


def mode_function(x):
    # If multiple modes, choose the first one
    return x.mode().iloc[0]

result = (IBdf.groupby(['route', 'booking_class', 'operating_carrier_code', 'marketing_carrier_code'])
          ['tier_points'].agg(mode_function).reset_index())
result = result[result['route']!='nan']
df = df.merge(result, on=['route', 'booking_class', 'operating_carrier_code', 'marketing_carrier_code'], how='left', suffixes=('', '_mode'))
df['tier_points'] = np.where(df['tier_points'] == 0, df['tier_points_mode'], df['tier_points'])
df.drop('tier_points_mode', axis=1, inplace=True)



df = df [['agl_party_id',
 'membership_txt',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'key_flight_date',
 'marketing_carrier_code',
 'operating_carrier_code',
 'route',
 'booking_cabin',
 'booking_class',
 'fare',
 'tax',
 'unified_rev',
 'MULTIPLIER_NO',
 'ff_tier_level',
 'JB_IND',
 'miles',
 'seg_haul_segment',
 'FARE_BASIS_CD',
 'BILLING_METHODOLOGY_NM',
 'FARE_TYP',
 'SBE_REGION_TXT',
 'accurate_exchange',
 'coupon_number',
 'origins',
 'booking_class_letter',
 'booking_cabin_letter',
 'zone_description',
 'route_name',
 'class_zone',
 'tier_points',
          'avios_earned',
         'psjs']]

In [113]:
######## good join will not create duplication
indicator = '1_merge'
# For df DataFrame
df['agl_party_id'] = df['agl_party_id'].astype(str)
df['route'] = df['route'].astype(str)

# For IBdf DataFrame
IBdf['agl_party_id'] = IBdf['agl_party_id'].astype(str)
IBdf['route'] = IBdf['route'].astype(str)

# Initial join
joined, didntjoinIB, didntjoinIAGL = custom_join(IBdf, df, ['agl_party_id', 'key_flight_date', 'ticket_number', 'route'],indicator)

print(joined.shape)


(5169351, 57)


In [114]:
didntjoinIAGL.dtypes

agl_party_id                       object
customer_id_best                   object
ff_identification                  object
matches                            object
ticket_number                      object
fare_family_description            object
key_flight_date            datetime64[ns]
route                              object
yq                                float32
total_payment                     float32
channel                            object
fare_basis                         object
avios_redeemed                     object
fare_class                         object
fare_product_name                  object
shuttle_indicator                  object
corp_fares                         object
bz_fares                           object
inclusive_fares                    object
agency_fares                       object
hidden_fares                       object
membership_txt                     object
scheme                             object
country_cd                        

In [122]:
indicator = '2_merge'
# Join the rows that didn't join from both dataframes
newly_joined, didntjoinIB, didntjoinIAGL = custom_join(didntjoinIB, didntjoinIAGL, ['agl_party_id', 'pnr','pnr_creation_date', 'key_flight_date', 'route'],indicator)

# Concatenate the newly joined rows to the 'joined' dataframe
joined = pd.concat([joined, newly_joined], ignore_index=True)
print(joined.shape)
result = pd.concat([joined, didntjoinIB, didntjoinIAGL], ignore_index=True)


(5169351, 58)


In [132]:
result3 = pd.concat([joined, didntjoinIAGL], ignore_index=True)

In [133]:
result3.shape

(6480580, 58)

In [135]:
result3.columns.to_list()

['agl_party_id',
 'membership_txt',
 'customer_id_best',
 'ff_tier_level',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'coupon_number',
 'fare_family_description',
 'key_flight_date',
 'route',
 'fare',
 'yq',
 'total_payment',
 'unified_rev',
 'channel',
 'fare_basis',
 'marketing_carrier_code',
 'operating_carrier_code',
 'seg_haul_segment',
 'avios_earned',
 'tier_points',
 'avios_redeemed',
 'booking_cabin',
 'miles',
 'fare_class',
 'fare_product_name',
 'booking_class_letter',
 'booking_class',
 'booking_cabin_letter',
 'zone_description',
 'shuttle_indicator',
 'route_name',
 'class_zone',
 'corp_fares',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'psjs',
 'tax',
 'MULTIPLIER_NO',
 'JB_IND',
 'FARE_BASIS_CD',
 'BILLING_METHODOLOGY_NM',
 'FARE_TYP',
 'SBE_REGION_TXT',
 'accurate_exchange',
 'ta']

In [136]:
result3 = result3[['agl_party_id',
 'membership_txt',
 'customer_id_best',
 'ff_tier_level',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'coupon_number',
 'fare_family_description',
 'key_flight_date',
 'route',
 'fare',
 'yq',
 'total_payment',
 'unified_rev',
 'channel',
 'fare_basis',
 'FARE_BASIS_CD',
 'marketing_carrier_code',
 'operating_carrier_code',
 'seg_haul_segment',
 'avios_earned',
 'tier_points',
 'avios_redeemed',
 'booking_cabin',
 'miles',
 'fare_class',
 'fare_product_name',
 'booking_class_letter',
 'booking_class',
 'booking_cabin_letter',
 'zone_description',
 'shuttle_indicator',
 'route_name',
 'class_zone',
 'corp_fares',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'psjs',
 'tax',
 'MULTIPLIER_NO',
 'JB_IND']]

In [137]:
result3 = result3[~(result3['agl_party_id'].isna()&result3['customer_id_best'].isna())]

In [138]:
result3.shape

(6480580, 53)

In [141]:
for column in result3:
    if result3[column].dtype == "float64":
        result3[column]=pd.to_numeric(result3[column], downcast="float")
    if result3[column].dtype == "int64":
        result3[column]=pd.to_numeric(result3[column], downcast="integer")

bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IB_cleanest.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
s3.to_csv(result3, s3_location, index=False)

{'paths': ['s3://gdp-dev-iagl-data-sagemaker/PHXTierModelling/2023_data/IB_dirty/IB_cleanest.csv'],
 'partitions_values': {}}

In [142]:
result3.dtypes

agl_party_id                       object
membership_txt                     object
customer_id_best                   object
ff_tier_level                      object
scheme                             object
country_cd                         object
market                             object
cust_birth_date            datetime64[ns]
cust_join_date             datetime64[ns]
gender                             object
ff_identification                  object
origins                            object
matches                            object
ticket_number                      object
pnr                                object
pnr_creation_date          datetime64[ns]
coupon_number                     float32
fare_family_description            object
key_flight_date            datetime64[ns]
route                              object
fare                              float32
yq                                float32
total_payment                     float32
unified_rev                       

In [ ]:
# bucket ='gdp-dev-iagl-data-sagemaker'
# folder = 'PHXTierModelling/2023_data/IB_dirty'
# filename = 'IBonlybeforeID.csv'

# s3_location = f's3://{bucket}/{folder}/{filename}'
# s3.to_csv(didntjoinIB, s3_location, index=False)

In [ ]:
didntjoinIB.dtypes

In [ ]:
# bucket ='gdp-dev-iagl-data-sagemaker'
# folder = 'PHXTierModelling/2023_data/IB_dirty'
# filename = 'IBonlyafterID.csv'

# s3_location = f's3://{bucket}/{folder}/{filename}'
# s3.to_csv(didntjoinIB, s3_location, index=False)

In [ ]:
joined.shape

In [ ]:
# bucket ='gdp-dev-iagl-data-sagemaker'
# folder = 'PHXTierModelling/2023_data/IB_dirty'
# filename = 'IAGLonly.csv'

# s3_location = f's3://{bucket}/{folder}/{filename}'
# s3.to_csv(didntjoinIAGL, s3_location, index=False)

In [ ]:
# bucket ='gdp-dev-iagl-data-sagemaker'
# folder = 'PHXTierModelling/2023_data/IB_dirty'
# filename = 'joins2.csv'

# s3_location = f's3://{bucket}/{folder}/{filename}'
# s3.to_csv(result, s3_location, index=False)

In [156]:

# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IB_cleanest.csv'

dtypes = {
    'agl_party_id': 'object',
    'membership_txt': 'object',
    'customer_id_best': 'object',
    'ff_tier_level': 'object',
    'scheme': 'object',
    'country_cd': 'object',
    'market': 'object',
    'cust_birth_date': 'object',
    'cust_join_date': 'object',
    'gender': 'object',
    'ff_identification': 'object',
    'origins': 'object',
    'matches': 'object',
    'ticket_number': 'object',
    'pnr': 'object',
    'pnr_creation_date': 'object',
    'coupon_number': 'float32',
    'fare_family_description': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'fare': 'float32',
    'yq': 'float32',
    'total_payment': 'float32',
    'unified_rev': 'float32',
    'channel': 'object',
    'fare_basis': 'object',
    'FARE_BASIS_CD': 'float32',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'seg_haul_segment': 'object',
    'avios_earned': 'float32',
    'tier_points': 'float32',
    'avios_redeemed': 'object',
    'booking_cabin': 'object',
    'miles': 'float32',
    'fare_class': 'object',
    'fare_product_name': 'object',
    'booking_class_letter': 'object',
    'booking_class': 'object',
    'booking_cabin_letter': 'object',
    'zone_description': 'object',
    'shuttle_indicator': 'object',
    'route_name': 'object',
    'class_zone': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'psjs': 'float32',
    'tax': 'float32',
    'MULTIPLIER_NO': 'float32',
    'JB_IND': 'object'
}


s3_location = f's3://{bucket}/{folder}/{filename}'
df = s3.read_csv(s3_location,dtype=dtypes,low_memory=False)
df['Customer Id'] = df['agl_party_id'].fillna(df['customer_id_best']+str('181818'))

In [157]:
df.columns.to_list()

['agl_party_id',
 'membership_txt',
 'customer_id_best',
 'ff_tier_level',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'coupon_number',
 'fare_family_description',
 'key_flight_date',
 'route',
 'fare',
 'yq',
 'total_payment',
 'unified_rev',
 'channel',
 'fare_basis',
 'FARE_BASIS_CD',
 'marketing_carrier_code',
 'operating_carrier_code',
 'seg_haul_segment',
 'avios_earned',
 'tier_points',
 'avios_redeemed',
 'booking_cabin',
 'miles',
 'fare_class',
 'fare_product_name',
 'booking_class_letter',
 'booking_class',
 'booking_cabin_letter',
 'zone_description',
 'shuttle_indicator',
 'route_name',
 'class_zone',
 'corp_fares',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'psjs',
 'tax',
 'MULTIPLIER_NO',
 'JB_IND',
 'Customer Id']

In [158]:
df.fare_product_name.value_counts()

Basic       487440
Optima      216545
Flexible     28515
Comfort      24701
Name: fare_product_name, dtype: int64

In [159]:
df.fare_family_description.value_counts()

BÁSICA       2704320
CLASICA      1251028
NO APLICA     744666
FLEXIBLE      432514
Name: fare_family_description, dtype: int64

In [160]:
df['Customer Id'].value_counts()

38694911     293
38710972     272
36284172     261
107960245    254
34561026     228
            ... 
34457282       1
32629056       1
32629075       1
31917447       1
115667768      1
Name: Customer Id, Length: 1221172, dtype: int64

In [170]:
def process_fares(df):
    # Create a new column for the results
    df['Fare Product'] = 'Unknown'

    # For carriers IB, I0, I2, YW
    mask_carrier = df['operating_carrier_code'].isin(['IB','I0','I2','YW'])
    df.loc[mask_carrier & (df['fare_product_name'] == "Optima") & (df['fare_family_description'] == "FLEXIBLE"), 'Fare Product'] = "Optima Flex"
    df.loc[mask_carrier & (df['fare_product_name'] == "Optima") & (df['fare_family_description'] != "FLEXIBLE"), 'Fare Product'] = "Standard"
    df.loc[mask_carrier & (df['fare_product_name'] == "Comfort"), 'Fare Product'] = "Comfort"
    df.loc[mask_carrier & (df['fare_product_name'] == "Basic"), 'Fare Product'] = "Basic"
    df.loc[mask_carrier & (df['fare_product_name'] == "Flexible"), 'Fare Product'] = "Flex"

    # For fare_family_description mappings
    fare_family_mapping = {
        'BÁSICA': 'Basic',
        'CLASICA': 'Standard',
        'FLEXIBLE': 'Flex'
    }
    df['Fare Product'] = df['Fare Product'].where(df['Fare Product'] != 'Unknown', df['fare_family_description'].map(fare_family_mapping).fillna('Unknown'))

    return df



# Read file from s3
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IB_cleanest.csv'

dtypes = {
    'agl_party_id': 'object',
    'membership_txt': 'object',
    'customer_id_best': 'object',
    'ff_tier_level': 'object',
    'scheme': 'object',
    'country_cd': 'object',
    'market': 'object',
    'cust_birth_date': 'object',
    'cust_join_date': 'object',
    'gender': 'object',
    'ff_identification': 'object',
    'origins': 'object',
    'matches': 'object',
    'ticket_number': 'object',
    'pnr': 'object',
    'pnr_creation_date': 'object',
    'coupon_number': 'float32',
    'fare_family_description': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'fare': 'float32',
    'yq': 'float32',
    'total_payment': 'float32',
    'unified_rev': 'float32',
    'channel': 'object',
    'fare_basis': 'object',
    'FARE_BASIS_CD': 'float32',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'seg_haul_segment': 'object',
    'avios_earned': 'float32',
    'tier_points': 'float32',
    'avios_redeemed': 'object',
    'booking_cabin': 'object',
    'miles': 'float32',
    'fare_class': 'object',
    'fare_product_name': 'object',
    'booking_class_letter': 'object',
    'booking_class': 'object',
    'booking_cabin_letter': 'object',
    'zone_description': 'object',
    'shuttle_indicator': 'object',
    'route_name': 'object',
    'class_zone': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'psjs': 'float32',
    'tax': 'float32',
    'MULTIPLIER_NO': 'float32',
    'JB_IND': 'object'
}


s3_location = f's3://{bucket}/{folder}/{filename}'
df = s3.read_csv(s3_location,dtype=dtypes,low_memory=False)
df['Customer Id'] = df['agl_party_id'].fillna(df['customer_id_best']+str('181818'))


df = process_fares(df)

condition = df['tier_points'].isna()
num_missing_tier_points = len(df.loc[condition])
print("Number of rows with missing Old Tier Points:", num_missing_tier_points)


df.replace(to_replace='^\s+$', value=np.nan, regex=True, inplace=True)
# List of columns to convert to numeric
columns_to_convert = ['ticket_number', 'coupon_number', 'fare', 'yq', 'total_payment',
                      'unified_rev', 'avios_earned', 'tier_points', 'avios_redeemed', 
                      'miles', 'psjs']

# Convert columns to numeric
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
# Replace empty strings with null values
df = df.replace('', np.nan)

df['Seg Business Lesiure'] = np.where((df['bz_fares'] == 1) | (df['corp_fares'] == 1), 'B', 'L') #making a bus/leis ind

rename_dict = {
    'key_flight_date':'Key Flight Date', 'route':'Route', 'fare':'Fare', 'yq':'Yq', 
    'miles':'distanceKM', 'channel':'Channel', 'operating_carrier_code':'Operating Carrier Code',
    'marketing_carrier_code':'Marketing Carrier Code', 'seg_haul_segment':'Seg Haul Segment',
    'ff_tier_level':'Tier', 'code_class_description': 'Booking Class Name',
    'avios_earned':'Avios Earned', 'tier_points':'Old Tier Points',
    'psjs':'Psjs', 'fare_family_description':'Fare Family', 'corp_fares':'corp_fare_ind',
    'unified_rev':'Unified Rev', 'booking_cabin':'Booking Cabin', 'shuttle_indicator' : 'MADBCN Shuttle',
    'fare_class': 'Fare Class Letter', 'booking_class':'Booking Class', 'zone_description':'IB Fare Zone'
}
df.rename(columns=rename_dict, inplace=True)

df.loc[df['Tier'].isin(['NO FF', 'UNKNOWN']), 'Tier'] = np.nan
df['distanceKM'] = df['distanceKM']/miles #convert from miles into KM (we set miles to be called distanceKM earlier in code)

#filter out extra dates first to reduce processing time
df['Key Flight Date'] = pd.to_datetime(df['Key Flight Date'])
df['Booking Cabin'] = df['Booking Cabin'].fillna('Unknown')

#these are all Vueling Flights so must be in economy
condition = (df['Booking Cabin'] == 'Unknown') & (df['Old Tier Points'] > 0)
df.loc[condition, 'Booking Cabin'] = 'Economy'


df['Sauce'], df['scheme'] = 'IBPLU', 'IBPLU'
df['Old Tier Points'].fillna(0, inplace=True)

#getting rid of spaces in main carriers so can use == and isin, tableau will remove spaces for rest, str.strip was slower (maybe retest??)#########################
#df['Operating Carrier Code'] = df['Operating Carrier Code'].str.strip()

df['Operating Carrier Code'].fillna('Unknown', inplace=True)
df['Marketing Carrier Code'].fillna('Unknown', inplace=True)
df.loc[df['Operating Carrier Code'].str.contains('JLP'), 'Operating Carrier Code'] = 'HLP'
df.loc[df['Marketing Carrier Code'].str.contains('JLP'), 'Marketing Carrier Code'] = 'HLP'

#Not Tier Points were awarded
condition = df['Old Tier Points'] == 0
num_rows_with_zero_tier_points = len(df.loc[condition])

print("Number of rows with Old Tier Points equal to 0 so no new tier points will be awarded:", num_rows_with_zero_tier_points)
print("proportion: ",num_rows_with_zero_tier_points/ len(df))


contains = ['BA', 'IB', 'I0', 'I2', 'VY', 'EI', 'YW', 'CJ', 'EZ', 'LM', 'AA', 'QR', 'JL', 'AY', 
            'AS', 'AT', 'CX', 'FJ', 'MH', 'QF', 'RJ', 'UL', 'WY', 'S7']
for code in contains:
    df['Operating Carrier Code'] = np.where(df['Operating Carrier Code'].str.contains(code), code, df['Operating Carrier Code'])
    df['Marketing Carrier Code'] = np.where(df['Marketing Carrier Code'].str.contains(code), code, df['Marketing Carrier Code'])
    
    
for column in df.select_dtypes(include=['float64']):
    df[column] = pd.to_numeric(df[column], downcast="float")
for column in df.select_dtypes(include=['int64']):
    df[column] = pd.to_numeric(df[column], downcast="integer")

# # Write file to s3
bucket ='gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data'
filename = 'IB_flights.csv'

s3_location = f's3://{bucket}/{folder}/{filename}'
s3.to_csv(df, s3_location, index=False)

Number of rows with missing Old Tier Points: 293867
Number of rows with Old Tier Points equal to 0 so no new tier points will be awarded: 7548
proportion:  0.0011647105660295837


{'paths': ['s3://gdp-dev-iagl-data-sagemaker/PHXTierModelling/2023_data/IB_flights.csv'],
 'partitions_values': {}}

In [171]:
df['Booking Cabin'].value_counts()

Economy         5232822
Business        1046655
First            114739
Prem Economy      86348
Unknown              16
Name: Booking Cabin, dtype: int64

In [172]:
df.Tier.value_counts()

CLASICA    3183171
PLATA       853683
ORO         775675
PLATINO     220628
Tier 1       30064
Tier 3       13200
Tier 2        9558
Tier 4        4711
Name: Tier, dtype: int64

In [ ]:
df

In [175]:
df.columns.to_list()

['agl_party_id',
 'membership_txt',
 'customer_id_best',
 'Tier',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'coupon_number',
 'Fare Family',
 'Key Flight Date',
 'Route',
 'Fare',
 'Yq',
 'total_payment',
 'Unified Rev',
 'Channel',
 'fare_basis',
 'FARE_BASIS_CD',
 'Marketing Carrier Code',
 'Operating Carrier Code',
 'Seg Haul Segment',
 'Avios Earned',
 'Old Tier Points',
 'avios_redeemed',
 'Booking Cabin',
 'distanceKM',
 'Fare Class Letter',
 'Fare Product',
 'booking_class_letter',
 'Booking Class',
 'booking_cabin_letter',
 'IB Fare Zone',
 'MADBCN Shuttle',
 'route_name',
 'class_zone',
 'corp_fare_ind',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'Psjs',
 'tax',
 'MULTIPLIER_NO',
 'JB_IND',
 'Customer Id',
 'Fare Product',
 'Seg Business Lesiure',
 'Sauce']

In [174]:
IBdf.columns.to_list()

['agl_party_id',
 'membership_txt',
 'customer_id_best',
 'ff_tier_level',
 'scheme',
 'country_cd',
 'market',
 'cust_birth_date',
 'cust_join_date',
 'gender',
 'ff_identification',
 'origins',
 'matches',
 'ticket_number',
 'pnr',
 'pnr_creation_date',
 'coupon_number',
 'fare_family_description',
 'key_flight_date',
 'route',
 'fare',
 'yq',
 'total_payment',
 'unified_rev',
 'channel',
 'fare_basis',
 'marketing_carrier_code',
 'operating_carrier_code',
 'seg_haul_segment',
 'avios_earned',
 'tier_points',
 'avios_redeemed',
 'booking_cabin',
 'miles',
 'fare_class',
 'fare_product_name',
 'booking_class_letter',
 'booking_class',
 'booking_cabin_letter',
 'zone_description',
 'shuttle_indicator',
 'route_name',
 'class_zone',
 'corp_fares',
 'bz_fares',
 'inclusive_fares',
 'agency_fares',
 'hidden_fares',
 'psjs']

In [2]:
bucket = 'gdp-dev-iagl-data-sagemaker'
folder = 'PHXTierModelling/2023_data/IB_dirty'
filename = 'IB_cleanest.csv'

dtypes = {
    'agl_party_id': 'object',
    'membership_txt': 'object',
    'customer_id_best': 'object',
    'ff_tier_level': 'object',
    'scheme': 'object',
    'country_cd': 'object',
    'market': 'object',
    'cust_birth_date': 'object',
    'cust_join_date': 'object',
    'gender': 'object',
    'ff_identification': 'object',
    'origins': 'object',
    'matches': 'object',
    'ticket_number': 'object',
    'pnr': 'object',
    'pnr_creation_date': 'object',
    'coupon_number': 'float32',
    'fare_family_description': 'object',
    'key_flight_date': 'object',
    'route': 'object',
    'fare': 'float32',
    'yq': 'float32',
    'total_payment': 'float32',
    'unified_rev': 'float32',
    'channel': 'object',
    'fare_basis': 'object',
    'FARE_BASIS_CD': 'float32',
    'marketing_carrier_code': 'object',
    'operating_carrier_code': 'object',
    'seg_haul_segment': 'object',
    'avios_earned': 'float32',
    'tier_points': 'float32',
    'avios_redeemed': 'object',
    'booking_cabin': 'object',
    'miles': 'float32',
    'fare_class': 'object',
    'fare_product_name': 'object',
    'booking_class_letter': 'object',
    'booking_class': 'object',
    'booking_cabin_letter': 'object',
    'zone_description': 'object',
    'shuttle_indicator': 'object',
    'route_name': 'object',
    'class_zone': 'object',
    'corp_fares': 'object',
    'bz_fares': 'object',
    'inclusive_fares': 'object',
    'agency_fares': 'object',
    'hidden_fares': 'object',
    'psjs': 'float32',
    'tax': 'float32',
    'MULTIPLIER_NO': 'float32',
    'JB_IND': 'object'
}


s3_location = f's3://{bucket}/{folder}/{filename}'
df = s3.read_csv(s3_location,dtype=dtypes,low_memory=False)
df['Customer Id'] = df['agl_party_id'].fillna(df['customer_id_best']+str('181818'))

NameError: name 's3' is not defined